# Using dependency relations for intent recognition

After extracting the entities, we want to find out what sort of intent the user carries – to book a flight, to purchase a meal on their already booked flight, cancel their flight, and so on. If you look at the intents list again, you will see that every intent includes a verb (to book) and an object that the verb acts on (flight, hotel, meal). Therefore, our first goal is to extract the transitive verb and the direct object of the verb.

## Linguistic primer

First we need to review some linguistic concepts related to sentence structure, including verbs and verb-object relations.

### Transitive verbs

A verb is a very important component of the sentence as it indicates the action in the sentence. The object of the sentence is the thing/person that is affected by the action of the verb. Hence, there's a natural connection between the sentence verb and objects. The concept of transitivity captures verb-object relations. A transitive verb is a verb that needs an object to act upon. Let's see some examples:

```
I bought flowers.
He loved his cat.
He borrowed my book.
```

In these example sentences, `bought`, `loved`, and `borrowed` are transitive verbs. In the first sentence, `bought` is the transitive verb and `flowers` is its object, the thing that has been bought by the sentence subject, `I`. We'll focus on the first sentence again. What happens if we erase the `flowers` object?

```
I bought
```

Bought **what?** Without an object, this sentence doesn't carry any meaning at all. In the preceding sentences, each of the objects completes the meaning of the verb. This is a way of understanding whether a verb is transitive not – erase the object and check whether the sentence remains semantically intact.

Some verbs are transitive and some verbs are intransitive. An **intransitive verb** is the opposite of a transitive verb; it doesn't need an object to act upon. Let's see some examples:

```
Yesterday I slept for 8 hours.
The cat ran towards me.
When I went out, the sun was shining.
Her cat died 3 days ago.
```

In all the preceding sentences, the verbs make sense without an object. If we erase all the words other than the subject and object, these sentences are still meaningful:

```
I slept.
The cat ran.
The sun was shining.
Her cat died.
```

Pairing an intransitive verb with an object doesn't make sense. You can't run someone/something, you can't shine something/someone, and you certainly cannot die something/someone.

### Sentence object

As we remarked before, the object is the thing/person that is affected by the verb's action. The action stated by the verb is committed by the sentence subject and the sentence object gets affected.

A sentence can be direct or indirect. A **direct object** answers the questions **whom?**/**what?**. You can find the direct object by asking: **The subject {verb} what/who?**. Here are some examples:

```
I bought flowers. I bought what? - flowers
He loved his cat. He loved who? - his cat
He borrowed my book. He borrowed what? - my book
```

An **indirect object** answers the questions **for what?**/**for whom?**/**to whom?**. Let's see some examples:
```
He gave me his book. He gave his book to whom? - me
He gave his book to me. He gave his book to whom? - me
```

Indirect objects are often preceded by the prepositions `to`, `for`, `from`, and so on. As you can see from these examples, an indirect object is also an object and is affected by the verb's action, but its role in the sentence is a bit different. An indirect object is sometimes viewed as the recipient of the direct object.

## Extracting transitive verbs and their direct objects

While recognizing the intent, usually we apply these steps to a user utterance:
1. Splitting the sentence into tokens.
2. Dependency parsing is performed by spaCy. We walk the dependency tree to extract the tokens and relations that we're interested in, which are the verb and the direct object.

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_md')
doc = nlp('find a flight from washington to sf')
displacy.render(doc, style='dep')

In this example sentence, the transitive verb is `find` and the direct object is `a flight`. The relation `dobj` connects a transitive verb to its direct object. If we follow the arc, semantically, we see that the user wants to commit the action of finding and the object they want to find is a flight. We can merge `find` and a `flight` into a single word, `findAflight` or `findFlight`, which can be this intent's name. Other intents can be `bookFlight`, `cancelFlight`, `bookMeal`, and so on.

Let's extract the verb and the direct object in a more systematic way. We'll first spot the direct object by looking for the dobj label in the sentence. To locate the transitive verb, we look at the direct object's syntactic head. A sentence can include more than one verb, hence we're careful while processing the verbs.

In [3]:
import spacy

nlp = spacy.load('en_core_web_md')
doc = nlp('find a flight from washington to sf')

for token in doc:
    if token.dep_ == 'dobj':
        print(token.head.text + token.text.capitalize())

findFlight


## Extracting multiple intents with conjunction relation

Some utterances carry multiple intents. For example, consider the following utterance from the corpus:

```
show all flights and fares from denver to san francisco
```

Here, the user wants to list all the flights and, at the same time, wants to see the fare info. One way of processing is considering these intents as a single and complex intent. In that case, we can express this complex intent as action: `show, objects: flights, fares`. Another and more common way of processing this sort of utterance is to label the utterance with multiple intents. In the dataset, this example utterance is marked with two intents as `atis_flight#atis_airfare`:

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_md')
doc = nlp('show all flights and fares from denver to san francisco')
displacy.render(doc, style='dep', options={'compact': True})

In the preceding diagram, we see that the `dobj` arc connects `show` and `flights`. The `conj` arc connects `flights` and `fares` to indicate the conjunction relation between them. The conjunction relation is built by a conjunction such as `and` or `or` and indicates that a noun is joined to another noun by this conjunction. In this situation, we extract the direct object
and its conjuncts.

In [16]:
import spacy
nlp = spacy.load('en_core_web_md')

dobj = None
doc = nlp('show all flights and fares from denver to san francisco')
for token in doc:
    if token.dep_ == 'dobj':
        dobj = token
        break
if dobj != None:
    conj = [t.text for t in token.conjuncts]
    verb = dobj.head
    print(verb, dobj, conj)

show flights ['fares']


## Recognizing the intent using wordlists

In some cases, tokens other than the transitive verb and the direct object contain the semantics of the user intent. In that case, you need to go further down in the syntactic relations and explore the sentence structure deeper.

As an example, consider the following utterance from our dataset:
```
i want to make a reservation for a flight
```

In this sentence, the verb-object pair that best describes the user intent is `want`-`flight`. However, if we look at the parse tree, we see that `want` and `flight` are not directly related in the parse tree. `want` is related to the transitive verb `make`, and `flight` is related to the direct object `reservation`, respectively:

In [17]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_md')
doc = nlp('i want to make a reservation for a flight')
displacy.render(doc, style='dep', options={'compact': True})

What will we do then? We can play a trick and keep a list of helper verbs such as `would like`, `want`, `make`, and `need`.

In [25]:
import spacy
nlp = spacy.load('en_core_web_md')

doc = nlp('i want to make a reservation for a flight')
dObj = None
tVerb = None

# Extract the direct object and its transitive verb
for token in doc:
    if token.dep_ == 'dobj':
        dObj = token
        tVerb = token.head

# Extract the intent verb
intentVerb = None
verbList = ['want', 'like', 'need', 'order']
if tVerb.text in verbList:
    intentVerb = tVerb
elif tVerb.head.dep_ == 'ROOT':
    intentVerb = tVerb.head

# Extract the object of the intent
intentObj = None
objList = ['flight', 'meal', 'booking']
if dObj.text in objList:
    intentObj = dObj
else:
    for child in dObj.children:
        if child.dep_ == 'prep':
            intentObj = list(child.children)[0]
            break
        elif child.dep_ == 'compound':
            intentObj = child
            break

if intentVerb != None and intentObj != None:
    print(intentVerb.text + intentObj.text.capitalize())

wantFlight
